In [4]:
import pytesseract

# If Tesseract is installed in a specific location
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\tesseract.exe"

In [5]:
pip install pytessaract

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pytessaract (from versions: none)
ERROR: No matching distribution found for pytessaract


In [2]:
import pandas as pd
from utils import download_images  # Assuming utils.py is in the same directory

# Load the dataset
train_file_path = r"C:\PM's Folder\Documents\Amazon ML Challenge Dataset\student_resource 3\Dataset2\train2.csv"
df = pd.read_csv(train_file_path)

# Extract the image links from the filtered dataset
image_links = df['image_link'].tolist()

# Define the download folder for images
download_folder = r"C:\PM's Folder\Documents\Amazon ML Challenge Dataset\images_voltage_wattage"

# Download the images
download_images(image_links, download_folder)

  0%|                                                                                        | 0/17221 [00:05<?, ?it/s]

KeyboardInterrupt



In [ ]:
import pandas as pd
import pytesseract
import cv2
import os
from PIL import Image
from pathlib import Path
import re
import numpy as np
from utils import download_images

entity_unit_map = {
    'voltage': {'kV', 'mV', 'V','volt','kilovolt','millivolt','Volt'},
    'wattage': {'kW', 'W','watt','kilowatt','Watt'},
    'width': {'centimetre', 'centimeter', 'foot', 'feet', 'inch', 'metre', 'meter', 'millimetre', 'millimeter', 'yard', 'cm', 'ft', 'in','m', '"', 'mm', 'yd', 'cms', 'mms', 'inches', 'meters', 'metres','centimetres', 'millimetres'},
    'depth': {'centimetre', 'centimeter', 'foot', 'feet', 'inch', 'metre','meter', 'millimetre', 'millimeter', 'yard', 'cm', 'ft', 'in', 'm', '"', 'mm', 'yd', 'cms', 'mms', 'inches', 'meters', 'metres','centimetres', 'millimetres'},
    'height': {'centimetre', 'centimeter', 'foot', 'feet', 'inch', 'metre','meter', 'millimetre', 'millimeter', 'yard', 'cm', 'ft', 'in', 
               'm', '"', 'mm', 'yd', 'cms', 'mms', 'inches', 'meters', 'metres', 
               'centimetres', 'millimetres'},
    'item_weight' : {
    'g', 'kg', 'µg', 'mg', 'oz', 'lb', 't', 'gram', 'kilogram', 'microgram', 
    'milligram', 'ounce', 'pound', 'ton', 'grams', 'kilograms', 'micrograms', 
    'milligrams', 'ounces', 'pounds', 'tons', 'gms', 'kgs','lbs', 'ozs', 
    'gm', 'kilogrammes', 'grammes', 'milligrammes', 'microgrammes','lbs'},
    'maximum_weight_recommendation': {
    'g', 'kg', 'µg', 'mg', 'oz', 'lb', 't', 'gram', 'kilogram', 'microgram', 
    'milligram', 'ounce', 'pound', 'ton', 'grams', 'kilograms', 'micrograms', 
    'milligrams', 'ounces', 'pounds', 'tons', 'gms', 'kgs','lbs', 'ozs', 
    'gm', 'kilogrammes', 'grammes', 'milligrammes', 'microgrammes','lbs'},
    'item_volume': {'L','mL','µL','cl','dl','ft','in','gal','cup','fl oz','pt','qt','centilitre','cubic foot','cubic inch','cup','decilitre','fluid ounce','gallon','imperial gallon','litre','microlitre','millilitre','pint','quart'}
}

# Load the training dataset
test_df = pd.read_csv(r"C:\PM's Folder\Documents\Amazon ML Challenge Dataset\student_resource 3\dataset\test.csv")

# Download images (assuming you have implemented the function in your utils.py)
download_folder = r"C:\PM's Folder\Documents\Amazon ML Challenge Dataset\images_test"
image_links = test_df['image_link'].tolist()  
download_images(image_links, download_folder)

# OCR function to extract text from image
import os
import cv2
import pytesseract

def ocr_image(image_path, psm=11, oem=1):
    try:
        # Ensure the image path is correctly formatted for Windows
        image_path = str(image_path)

        # Check if the image file exists before trying to read it
        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            return ""

        # Read the image
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Check if the image was successfully loaded
        if img is None:
            print(f"Error reading image: {image_path}")
            return ""

        # Apply OCR with custom PSM and OEM settings
        config = f'--psm {psm} --oem {oem}'
        text = pytesseract.image_to_string(img, config=config)
        return text
    except Exception as e:
        # Log the exception for debugging purposes
        print(f"Error in OCR for {image_path}: {e}")
        return ""

def convert_unit_to_words(value, unit, entity_name):
    unit_conversion_map = {
    'voltage': {
        'V': 'volt',
        'kV': 'kilovolt',
        'mV': 'millivolt',
        'volt': 'volt',
        'kilovolt': 'kilovolt',
        'millivolt': 'millivolt',
        'Volt': 'volt'
    },
    'wattage': {
        'W': 'watt',
        'kW': 'kilowatt',
        'watt': 'watt',
        'kilowatt': 'kilowatt',
        'Watt': 'watt'
    },
    'width': {
        'cm': 'centimetre',
        'm': 'metre',
        'mm': 'millimetre',
        'in': 'inch',
        'ft': 'foot',
        '"': 'inch',
        'yd': 'yard',
        'centimetre': 'centimetre',
        'centimeter': 'centimetre',
        'feet': 'foot',
        'inch': 'inch',
        'meter': 'metre',
        'millimetre': 'millimetre',
        'millimeter': 'millimetre',
        'yard': 'yard',
        'cms': 'centimetre',
        'mms': 'millimetre',
        'inches': 'inch',
        'meters': 'metre',
        'metres': 'metre',
        'centimetres': 'centimetre',
        'millimetres': 'millimetre'
    },
    'depth': {
        'cm': 'centimetre',
        'm': 'metre',
        'mm': 'millimetre',
        'in': 'inch',
        'ft': 'foot',
        '"': 'inch',
        'yd': 'yard',
        'centimetre': 'centimetre',
        'centimeter': 'centimetre',
        'feet': 'foot',
        'inch': 'inch',
        'meter': 'metre',
        'millimetre': 'millimetre',
        'millimeter': 'millimetre',
        'yard': 'yard',
        'cms': 'centimetre',
        'mms': 'millimetre',
        'inches': 'inch',
        'meters': 'metre',
        'metres': 'metre',
        'centimetres': 'centimetre',
        'millimetres': 'millimetre'
    },
    'height': {
        'cm': 'centimetre',
        'm': 'metre',
        'mm': 'millimetre',
        'in': 'inch',
        'ft': 'foot',
        '"': 'inch',
        'yd': 'yard',
        'centimetre': 'centimetre',
        'centimeter': 'centimetre',
        'feet': 'foot',
        'inch': 'inch',
        'meter': 'metre',
        'millimetre': 'millimetre',
        'millimeter': 'millimetre',
        'yard': 'yard',
        'cms': 'centimetre',
        'mms': 'millimetre',
        'inches': 'inch',
        'meters': 'metre',
        'metres': 'metre',
        'centimetres': 'centimetre',
        'millimetres': 'millimetre'
    },
    'item_weight': {
        'g': 'gram',
        'kg': 'kilogram',
        'µg': 'microgram',
        'mg': 'milligram',
        'oz': 'ounce',
        'lb': 'pound',
        't': 'ton',
        'gram': 'gram',
        'kilogram': 'kilogram',
        'microgram': 'microgram',
        'milligram': 'milligram',
        'ounce': 'ounce',
        'pound': 'pound',
        'ton': 'ton',
        'grams': 'gram',
        'kilograms': 'kilogram',
        'micrograms': 'microgram',
        'milligrams': 'milligram',
        'ounces': 'ounce',
        'pounds': 'pound',
        'tons': 'ton',
        'gms': 'gram',
        'kgs': 'kilogram',
        'lbs': 'pound',
        'ozs': 'ounce',
        'gm': 'gram',
        'kilogrammes': 'kilogram',
        'grammes': 'gram',
        'milligrammes': 'milligram',
        'microgrammes': 'microgram',
        'lbs': 'pound'
    },
    'maximum_weight_recommendation': {
        'g': 'gram',
        'kg': 'kilogram',
        'µg': 'microgram',
        'mg': 'milligram',
        'oz': 'ounce',
        'lb': 'pound',
        't': 'ton',
        'gram': 'gram',
        'kilogram': 'kilogram',
        'microgram': 'microgram',
        'milligram': 'milligram',
        'ounce': 'ounce',
        'pound': 'pound',
        'ton': 'ton',
        'grams': 'gram',
        'kilograms': 'kilogram',
        'micrograms': 'microgram',
        'milligrams': 'milligram',
        'ounces': 'ounce',
        'pounds': 'pound',
        'tons': 'ton',
        'gms': 'gram',
        'kgs': 'kilogram',
        'lbs': 'pound',
        'ozs': 'ounce',
        'gm': 'gram',
        'kilogrammes': 'kilogram',
        'grammes': 'gram',
        'milligrammes': 'milligram',
        'microgrammes': 'microgram',
        'lbs': 'pound'
    },
    'item_volume': {
        'L': 'litre',
        'mL': 'millilitre',
        'µL': 'microlitre',
        'cl': 'centilitre',
        'dl': 'decilitre',
        'ft': 'cubic foot',
        'in': 'cubic inch',
        'gal': 'gallon',
        'cup': 'cup',
        'fl oz': 'fluid ounce',
        'pt': 'pint',
        'qt': 'quart',
        'centilitre': 'centilitre',
        'cubic foot': 'cubic foot',
        'cubic inch': 'cubic inch',
        'decilitre': 'decilitre',
        'fluid ounce': 'fluid ounce',
        'gallon': 'gallon',
        'imperial gallon': 'imperial gallon',
        'litre': 'litre',
        'microlitre': 'microlitre',
        'millilitre': 'millilitre',
        'pint': 'pint',
        'quart': 'quart'
    }
}
    
    # Get the conversion map for the given entity_name
    conversion_map = unit_conversion_map.get(entity_name, {})
    
    # Convert the unit if it exists in the conversion map
    word_unit = conversion_map.get(unit, unit)  # Fallback to original unit if not found
    
    # Return the value with the converted unit
    return f"{value} {word_unit}"

# Function to extract number and unit from OCR output
def extract_entity_value(text, entity_name):
    
    # Fetch allowed units for the given entity
    allowed_units = entity_unit_map[entity_name]
    
    # Regular expression to capture ranges and single values with units
    # Exclude alphanumeric patterns before the hyphen to avoid interpreting them as ranges
    pattern = re.compile(r'(?<![a-zA-Z])(\d+(\.\d+)?)\s*-\s*(\d+(\.\d+)?)\s*([a-zA-Z]+)|(\d+(\.\d+)?)\s*([a-zA-Z"\'\u00B0]+)')
    
    # Find all matches in the input text
    matches = pattern.findall(text)
    
    # Iterate through the matches to extract and return the appropriate value
    for match in matches:
        # Check if it's a range of values (make sure both values are numeric)
        if match[2]:  # Range found
            value1 = float(match[0])
            value2 = float(match[2])
            unit1 = match[4]
            
            # Ensure the unit is in the allowed units list
            if unit1 in allowed_units:
                return f"[{value1},{value2}] {unit1}"
        
        # Check if it's a single value
        elif match[5]:  # Single value found
            value1 = float(match[5])
            unit1 = match[7]
            
            # Ensure the unit is in the allowed units list
            if unit1 in allowed_units:
                return f"{value1} {unit1}"
    
    # If no valid value or unit is found, return None
    if entity_name == "voltage":
        return '0 volt'
    elif entity_name == "wattage":
        return '0 watt'
    elif entity_name == "width":
        return '0 cm'
    elif entity_name == "depth":
        return '0 cm'
    elif entity_name == "height":
        return '0 cm'
    elif entity_name == "item_weight":
        return '0 gram'
    elif entity_name == "maximum_weight_recommendation":
        return '0 gram'
    elif entity_name == "item_volume":
        return '0 L'

# Apply OCR to extract entity values for training dataset
test_df['prediction'] = ""

for idx, row in test_df.iterrows():
    image_link = row['image_link']
    
    if pd.isna(image_link) or not isinstance(image_link, str):
        # Skip rows with missing or invalid image links
        print(f"Invalid image link at index {idx}")
        continue

    # Properly construct the image path
    image_filename = Path(image_link).name  # Extract the image file name from the link
    image_path = Path(download_folder) / image_filename  # Full path to the image

    # Check if the image exists
    if not image_path.exists():
        print(f"Image not found at {image_path} for index {idx}")
        continue

    # Apply OCR to the image
    entity_name = row['entity_name']  # Example: item_weight, voltage, etc.
    text = ocr_image(image_path)

    # Extract the predicted entity value (assuming the function `extract_entity_value` is defined)
    prediction = extract_entity_value(text, entity_name)

    # Convert the unit to word format and store in the DataFrame
    value, unit = prediction.split()  # Split the value and unit
    prediction_word_format = convert_unit_to_words(value, unit, entity_name)
    test_df.at[idx, 'prediction'] = prediction_word_format
    print(f"Index: {idx}, Prediction: {prediction_word_format}")

# Save the predictions
test_df[['prediction']].to_csv(r"C:\PM's Folder\Documents\train_predictions.csv", index=True) 

100%|████████████████████████████████████████████████████████████████████████| 131187/131187 [00:26<00:00, 4859.56it/s]


Index: 0, Prediction: 0 centimetre
Index: 1, Prediction: 42.0 centimetre
Index: 2, Prediction: 42.0 centimetre
Index: 3, Prediction: 42.0 centimetre
Index: 4, Prediction: 10.5 centimetre
Index: 5, Prediction: 10.5 centimetre
Index: 6, Prediction: 10.5 centimetre
Index: 7, Prediction: 0 centimetre
Index: 8, Prediction: 40.0 centimetre
Index: 9, Prediction: 40.0 centimetre
Index: 10, Prediction: 0 gram
Index: 11, Prediction: 0 centimetre
Index: 12, Prediction: 0 centimetre
Index: 13, Prediction: 0 centimetre
Index: 14, Prediction: 0 centimetre
Index: 15, Prediction: 0 gram
Index: 16, Prediction: 0 gram
Index: 17, Prediction: 0 gram
Index: 18, Prediction: 0 watt
Index: 19, Prediction: 0 volt
Index: 20, Prediction: 0 centimetre
Index: 21, Prediction: 0 centimetre
Index: 22, Prediction: 0 litre
Index: 23, Prediction: 32.0 centimetre
Index: 24, Prediction: 32.0 centimetre
Index: 25, Prediction: 0 centimetre
Index: 26, Prediction: 0 centimetre
Index: 27, Prediction: 0 gram
Index: 28, Predicti

In [8]:
# Save the predictions for the first 500 rows
train_df['index', 'prediction']].to_csv(r"C:\PM's Folder\Documents\train_predictions.csv", index=False) 
# Save the result for training set
train_df[['prediction']].to_csv(r"C:\PM's Folder\Documents\Amazon ML Challenge Dataset\train_predictions.csv", index=True)

# Load the test dataset (without 'entity_value')
test_df = pd.read_csv(r"C:\PM's Folder\Documents\Amazon ML Challenge Dataset\student_resource 3\Dataset2\test2.csv")

# Download images for the test set
test_image_links = test_df['image_link'].tolist()
download_images(test_image_links, download_folder)

# Apply OCR and extract predictions for test dataset
test_df['prediction'] = ""
for idx, row in test_df.iterrows():
    image_path = Path(download_folder) / Path(row['image_link']).name
    entity_name = row['entity_name']  # Example: item_weight, voltage, etc.
    text = ocr_image(image_path)
    prediction = extract_entity_value(text, entity_name)
    test_df.at[idx, 'prediction'] = prediction

# Save the predictions for the test set
test_df[['index', 'prediction']].to_csv('/mnt/data/test_predictions.csv', index=False)

 47%|███████████████████████████████████▉                                         | 5103/10935 [01:24<01:36, 60.62it/s]


KeyboardInterrupt: 